# Credit Markets Final Exam - Complete Solution
## FINM 35700 - Spring 2024
### UChicago Financial Mathematics

**Date:** May 3, 2024

This notebook contains complete solutions to all 6 problems in the Credit Markets final exam.

**Problems:**
1. Overall Understanding of Credit Models (40 points)
2. Risk and Scenario Analysis for AAPL Bond (20 points)
3. CDS Calibration and Pricing (20 points)
4. Derivation of Bond PVs and DV01s in sympy (25 points)
5. LQD ETF Basket Analysis - Bucketed DV01 Risks (25 points)
6. Nelson-Siegel Model for ORCL Curve (25 points)

**Total:** 155 points


In [ ]:
# Import required libraries
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt
from scipy.optimize import minimize
import sympy as sp
from matplotlib import cm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set plot display
%matplotlib inline

# Import credit market tools
import sys
sys.path.append('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam')
from credit_market_tools import *

# Set calculation date
calc_date = ql.Date(3, 5, 2024)
ql.Settings.instance().evaluationDate = calc_date
as_of_date = pd.to_datetime('2024-05-03')

# Data path
data_path = '/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/data/'

print(f'Calculation Date: {calc_date}')
print(f'All packages loaded successfully!')


# Problem 1

In [ ]:
PROBLEM 1: Overall Understanding of Credit Models (40 points)

print("\n" + "="*80)
print("PROBLEM 1: Overall Understanding of Credit Models (40 points)")

# Problem 1a: Fixed rate bond prices in the hazard rate model
print("\n### Problem 1a: Fixed rate bond prices in the hazard rate model ###")
print("\nFor a fixed rate bond in the hazard rate model:")
print("Formula: BondPV = Σ(c/2 * e^(-k*y/2)) + e^(-T*y) with default adjustment")
print()

answers_1a = {
    "1. Fixed rate bond price is decreasing in interest rate": "TRUE",
    "2. Fixed rate bond price is decreasing in hazard rate": "TRUE",
    "3. Fixed rate bond price is decreasing in expected recovery rate": "FALSE",
    "4. Fixed rate bond price is decreasing in coupon": "FALSE",
    "5. Fixed rate bond price is decreasing in bond maturity": "FALSE (generally)"
}

for q, a in answers_1a.items():
    print(f"{q}: {a}")

# Problem 1b: Fixed rate bond yields in the hazard rate model
print("\n### Problem 1b: Fixed rate bond yields in the hazard rate model ###")
print()

answers_1b = {
    "1. Fixed rate bond yield is decreasing in interest rate": "FALSE",
    "2. Fixed rate bond yield is decreasing in hazard rate": "FALSE",
    "3. Fixed rate bond yield is decreasing in expected recovery rate": "TRUE",
    "4. Fixed rate bond yield is independent of the coupon": "TRUE",
    "5. Fixed rate bond yield is decreasing in bond maturity": "FALSE (generally)"
}

for q, a in answers_1b.items():
    print(f"{q}: {a}")

# Problem 1c: Equity and equity volatility in Merton model
print("\n### Problem 1c: Equity and equity volatility in Merton Structural Credit Model ###")
print("\nEquity as a call option on assets with liabilities as strike")
print()

answers_1c = {
    "1. Equity value is decreasing with company assets": "FALSE",
    "2. Equity volatility is decreasing with company assets": "TRUE",
    "3. Equity value is decreasing with assets volatility": "FALSE",
    "4. Equity value is decreasing with company liabilities": "TRUE",
    "5. Equity volatility is decreasing with company liabilities": "FALSE"
}

for q, a in answers_1c.items():
    print(f"{q}: {a}")

# Problem 1d: Yield and expected recovery rate in Merton model
print("\n### Problem 1d: Yield and expected recovery rate in Merton Structural Credit Model ###")
print()

answers_1d = {
    "1. Yield is decreasing with company liabilities": "FALSE",
    "2. Expected recovery rate is decreasing with company liabilities": "TRUE",
    "3. Yield is decreasing with assets volatility": "FALSE",
    "4. Credit spread is decreasing with asset values": "TRUE",
    "5. Credit spread is decreasing with assets volatility": "FALSE"
}

for q, a in answers_1d.items():
    print(f"{q}: {a}")

print("\n" + "="*80)
print("Problem 1 Complete!")

# 

# Problem 2

In [ ]:
PROBLEM 2: Risk and Scenario Analysis for AAPL Bond (20 points)

print("\n" + "="*80)
print("PROBLEM 2: Risk and Scenario Analysis for AAPL Bond (20 points)")

# Problem 2a: Create AAPL bond object
print("\n### Problem 2a: Create the AAPL fixed-rate corporate bond object ###")

# Load bond symbology
bond_symbology_df = pd.read_excel(data_path + 'bond_symbology.xlsx')
print(f"\nLoaded bond symbology: {len(bond_symbology_df)} bonds")

# Find AAPL bond
aapl_bond_row = bond_symbology_df[bond_symbology_df['isin'] == 'US037833AT77'].iloc[0]
print(f"\nAAPL Bond Details:")
print(f"Security: {aapl_bond_row['security']}")
print(f"ISIN: {aapl_bond_row['isin']}")
print(f"FIGI: {aapl_bond_row['figi']}")
print(f"Coupon: {aapl_bond_row['coupon']}%")
print(f"Maturity: {aapl_bond_row['maturity']}")

# Create bond object
aapl_bond = create_bond_from_symbology(aapl_bond_row.to_dict())

# Get cashflows
aapl_cashflows = get_bond_cashflows(aapl_bond, calc_date)
print(f"\nAAPL Bond Cashflows:")
print(aapl_cashflows.to_string())

# Problem 2b: Compute bond price, DV01, duration, convexity
print("\n### Problem 2b: Compute bond price, DV01, duration and convexity ###")

# Load market data
bond_market_df = pd.read_excel(data_path + 'bond_market_prices_eod.xlsx')
bond_market_df['date'] = pd.to_datetime(bond_market_df['date'])
# Calculate mid price and mid yield
bond_market_df['mid_price'] = (bond_market_df['bidPrice'] + bond_market_df['askPrice']) / 2
bond_market_df['mid_yield'] = (bond_market_df['bidYield'] + bond_market_df['askYield']) / 2

aapl_market = bond_market_df[(bond_market_df['isin'] == 'US037833AT77') &
                              (bond_market_df['date'] == as_of_date)].iloc[0]

aapl_ytm = aapl_market['mid_yield'] / 100.0  # Convert to decimal
print(f"\nAAPL Bond Market Data (as of {as_of_date.date()}):")
print(f"Mid Yield: {aapl_ytm*100:.4f}%")

# Set up yield curve for pricing
flat_yield_curve = ql.FlatForward(calc_date, aapl_ytm, ql.Actual365Fixed())
yield_curve_handle = ql.YieldTermStructureHandle(flat_yield_curve)

# Create pricing engine
pricing_engine = ql.DiscountingBondEngine(yield_curve_handle)
aapl_bond.setPricingEngine(pricing_engine)

# Calculate metrics
aapl_price = aapl_bond.cleanPrice()
aapl_dv01 = -aapl_bond.cleanPrice() * aapl_bond.duration() / 10000
aapl_duration = ql.BondFunctions.duration(aapl_bond, aapl_ytm, ql.Actual365Fixed(),
                                          ql.Compounded, ql.Semiannual)
aapl_convexity = ql.BondFunctions.convexity(aapl_bond, aapl_ytm, ql.Actual365Fixed(),
                                            ql.Compounded, ql.Semiannual)

print(f"\nBond Metrics (Analytic Method):")
print(f"Price: {aapl_price:.4f}")
print(f"DV01: {aapl_dv01:.6f}")
print(f"Duration: {aapl_duration:.4f} years")
print(f"Convexity: {aapl_convexity:.4f}")

# Problem 2c: Scenario bond prices
print("\n### Problem 2c: Compute and plot scenario bond prices ###")

# Create yield scenarios
yield_scenarios = np.arange(0.02, 0.105, 0.005)
scenario_prices = []

for scenario_yield in yield_scenarios:
    flat_curve = ql.FlatForward(calc_date, scenario_yield, ql.Actual365Fixed())
    curve_handle = ql.YieldTermStructureHandle(flat_curve)
    engine = ql.DiscountingBondEngine(curve_handle)
    aapl_bond.setPricingEngine(engine)
    scenario_prices.append(aapl_bond.cleanPrice())

# Create plotly figure
fig_prices = go.Figure()
fig_prices.add_trace(go.Scatter(
    x=yield_scenarios * 100,
    y=scenario_prices,
    mode='lines+markers',
    name='Bond Price',
    line=dict(color='blue', width=2)
))
fig_prices.add_trace(go.Scatter(
    x=[aapl_ytm * 100],
    y=[aapl_price],
    mode='markers',
    name='Current Market',
    marker=dict(color='red', size=12, symbol='star')
))
fig_prices.update_layout(
    title='AAPL Bond Price vs Yield Scenarios',
    xaxis_title='Yield (%)',
    yaxis_title='Bond Price',
    hovermode='x unified'
)
fig_prices.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/aapl_scenario_prices.html')
print("\nScenario prices plot saved to aapl_scenario_prices.html")

# Problem 2d: Scenario durations and convexities
print("\n### Problem 2d: Compute and plot scenario durations and convexities ###")

scenario_durations = []
scenario_convexities = []

for scenario_yield in yield_scenarios:
    dur = ql.BondFunctions.duration(aapl_bond, scenario_yield, ql.Actual365Fixed(),
                                    ql.Compounded, ql.Semiannual)
    cvx = ql.BondFunctions.convexity(aapl_bond, scenario_yield, ql.Actual365Fixed(),
                                     ql.Compounded, ql.Semiannual)
    scenario_durations.append(dur)
    scenario_convexities.append(cvx)

# Plot durations
fig_dur = go.Figure()
fig_dur.add_trace(go.Scatter(
    x=yield_scenarios * 100,
    y=scenario_durations,
    mode='lines+markers',
    name='Duration',
    line=dict(color='green', width=2)
))
fig_dur.update_layout(
    title='AAPL Bond Duration vs Yield Scenarios',
    xaxis_title='Yield (%)',
    yaxis_title='Duration (years)',
    hovermode='x unified'
)
fig_dur.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/aapl_scenario_durations.html')

# Plot convexities
fig_cvx = go.Figure()
fig_cvx.add_trace(go.Scatter(
    x=yield_scenarios * 100,
    y=scenario_convexities,
    mode='lines+markers',
    name='Convexity',
    line=dict(color='purple', width=2)
))
fig_cvx.update_layout(
    title='AAPL Bond Convexity vs Yield Scenarios',
    xaxis_title='Yield (%)',
    yaxis_title='Convexity',
    hovermode='x unified'
)
fig_cvx.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/aapl_scenario_convexities.html')
print("Scenario duration and convexity plots saved")

print("\n" + "="*80)
print("Problem 2 Complete!")

# 

# Problem 3

In [ ]:
PROBLEM 3: CDS Calibration and Pricing (20 points)

print("\n" + "="*80)
print("PROBLEM 3: CDS Calibration and Pricing (20 points)")

# Problem 3a: Calibrate SOFR yield curve
print("\n### Problem 3a: Calibrate the US SOFR yield curve ###")

# Load SOFR swap data
sofr_symbology_df = pd.read_excel(data_path + 'sofr_swaps_symbology.xlsx')
sofr_market_df = pd.read_excel(data_path + 'sofr_swaps_market_data_eod.xlsx')
sofr_market_df['date'] = pd.to_datetime(sofr_market_df['date'])

# Filter for as_of_date
sofr_market_filtered = sofr_market_df[sofr_market_df['date'] == as_of_date].copy()

# Merge symbology and market data
sofr_combined = sofr_symbology_df.merge(sofr_market_filtered, on='figi')

print(f"\nSOFR Swaps Data: {len(sofr_combined)} instruments")

# Calibrate SOFR curve
sofr_curve = calibrate_sofr_curve_from_frame(calc_date, sofr_combined)
sofr_curve_handle = ql.YieldTermStructureHandle(sofr_curve)

# Get curve details
curve_dates = [calc_date + ql.Period(i, ql.Months) for i in range(1, 361)]
sofr_curve_df = get_yield_curve_details_df(sofr_curve, curve_dates)

print(f"\nSOFR Curve calibrated successfully")
print(f"Sample zero rates (first 5 tenors):")
print(sofr_curve_df.head())

# Plot SOFR curves
fig_sofr = make_subplots(
    rows=1, cols=2,
    subplot_titles=('SOFR Zero Interest Rates', 'SOFR Discount Factors')
)

fig_sofr.add_trace(
    go.Scatter(x=sofr_curve_df['TTM'], y=sofr_curve_df['Zero_Interest_Rate'] * 100,
               mode='lines', name='Zero Rate', line=dict(color='blue', width=2)),
    row=1, col=1
)

fig_sofr.add_trace(
    go.Scatter(x=sofr_curve_df['TTM'], y=sofr_curve_df['Discount_Factor'],
               mode='lines', name='Discount Factor', line=dict(color='green', width=2)),
    row=1, col=2
)

fig_sofr.update_xaxes(title_text='Time to Maturity (years)', row=1, col=1)
fig_sofr.update_xaxes(title_text='Time to Maturity (years)', row=1, col=2)
fig_sofr.update_yaxes(title_text='Zero Rate (%)', row=1, col=1)
fig_sofr.update_yaxes(title_text='Discount Factor', row=1, col=2)
fig_sofr.update_layout(height=400, showlegend=False, title_text='SOFR Yield Curve')
fig_sofr.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/sofr_curve.html')
print("SOFR curve plots saved")

# Problem 3b: Load and explore Ford CDS data
print("\n### Problem 3b: Load and explore CDS market data for Ford Motor Credit ###")

# Load CDS data
cds_market_df = pd.read_excel(data_path + 'cds_market_data_eod.xlsx')
cds_market_df['date'] = pd.to_datetime(cds_market_df['date'])

# Filter for Ford Motor Credit
ford_cds = cds_market_df[cds_market_df['ticker'] == 'F'].copy()
ford_cds = ford_cds.sort_values(['date', 'tenor'])

print(f"\nFord CDS Data: {len(ford_cds)} observations")
print(f"Date range: {ford_cds['date'].min()} to {ford_cds['date'].max()}")
print(f"Available tenors: {sorted(ford_cds['tenor'].unique())}")

# Get latest data
ford_latest = ford_cds[ford_cds['date'] == ford_cds['date'].max()].copy()
print(f"\nFord CDS Par Spreads (as of {ford_latest['date'].iloc[0].date()}):")
print(ford_latest[['tenor', 'par_spread_bps']].to_string(index=False))

# Plot historical CDS spreads
fig_ford_hist = go.Figure()
for tenor in sorted(ford_cds['tenor'].unique()):
    tenor_data = ford_cds[ford_cds['tenor'] == tenor]
    fig_ford_hist.add_trace(go.Scatter(
        x=tenor_data['date'],
        y=tenor_data['par_spread_bps'],
        mode='lines',
        name=f'{tenor}Y'
    ))

fig_ford_hist.update_layout(
    title='Ford CDS Par Spreads - Historical Time Series',
    xaxis_title='Date',
    yaxis_title='Par Spread (bps)',
    hovermode='x unified'
)
fig_ford_hist.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/ford_cds_historical.html')
print("Ford CDS historical plot saved")

# Problem 3c: Calibrate Ford hazard rate curve
print("\n### Problem 3c: Calibrate the Ford hazard rate curve ###")

# Prepare CDS par spreads for calibration
ford_as_of = ford_cds[ford_cds['date'] == as_of_date].copy()
ford_as_of = ford_as_of.sort_values('tenor')

cds_par_spreads = {}
for _, row in ford_as_of.iterrows():
    tenor_years = int(row['tenor'])
    cds_par_spreads[tenor_years] = row['par_spread_bps']

print(f"\nCDS Par Spreads for calibration:")
for tenor, spread in cds_par_spreads.items():
    print(f"  {tenor}Y: {spread:.2f} bps")

# Calibrate hazard rate curve (using 40% recovery rate)
recovery_rate = 0.4
ford_hazard_curve = calibrate_cds_hazard_rate_curve(
    calc_date, sofr_curve_handle, cds_par_spreads, recovery_rate
)

# Get hazard rate curve details
ford_hazard_df = get_hazard_rates_df(ford_hazard_curve)

print(f"\nFord Hazard Rate Curve calibrated successfully")
print(f"Sample hazard rates (first 5 tenors):")
print(ford_hazard_df.head())

# Plot hazard rates and survival probabilities
fig_ford_hz = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Ford Hazard Rates', 'Ford Survival Probabilities')
)

fig_ford_hz.add_trace(
    go.Scatter(x=ford_hazard_df['TTM'], y=ford_hazard_df['Hazard_Rate'] * 100,
               mode='lines', name='Hazard Rate', line=dict(color='red', width=2)),
    row=1, col=1
)

fig_ford_hz.add_trace(
    go.Scatter(x=ford_hazard_df['TTM'], y=ford_hazard_df['Survival_Probability'],
               mode='lines', name='Survival Prob', line=dict(color='orange', width=2)),
    row=1, col=2
)

fig_ford_hz.update_xaxes(title_text='Time to Maturity (years)', row=1, col=1)
fig_ford_hz.update_xaxes(title_text='Time to Maturity (years)', row=1, col=2)
fig_ford_hz.update_yaxes(title_text='Hazard Rate (%)', row=1, col=1)
fig_ford_hz.update_yaxes(title_text='Survival Probability', row=1, col=2)
fig_ford_hz.update_layout(height=400, showlegend=False, title_text='Ford Hazard Rate Curve')
fig_ford_hz.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/ford_hazard_curve.html')
print("Ford hazard rate curve plots saved")

# Problem 3d: CDS Valuation
print("\n### Problem 3d: CDS Valuation ###")

# Create CDS with 100 bps coupon and 2029-06-20 maturity
cds_coupon = 0.0100  # 100 bps
cds_maturity = ql.Date(20, 6, 2029)
cds_notional = 10_000_000  # $10MM

# Create CDS schedule
cds_schedule = ql.Schedule(
    calc_date,
    cds_maturity,
    ql.Period(ql.Quarterly),
    ql.UnitedStates(ql.UnitedStates.GovernmentBond),
    ql.Following,
    ql.Unadjusted,
    ql.DateGeneration.CDS,
    False
)

# Create CDS
ford_cds_instrument = ql.CreditDefaultSwap(
    ql.Protection.Buyer,
    cds_notional,
    cds_coupon,
    cds_schedule,
    ql.Following,
    ql.Actual360(),
    True,
    True,
    calc_date,
    ql.Actual360(True),
    True,
    ql.Date()
)

# Create pricing engine
cds_engine = ql.MidPointCdsEngine(
    ql.DefaultProbabilityTermStructureHandle(ford_hazard_curve),
    recovery_rate,
    sofr_curve_handle
)
ford_cds_instrument.setPricingEngine(cds_engine)

# Calculate CDS metrics
cds_npv = ford_cds_instrument.NPV()
cds_default_leg = ford_cds_instrument.defaultLegNPV()
cds_coupon_leg = ford_cds_instrument.couponLegNPV()
cds_fair_spread = ford_cds_instrument.fairSpread() * 10000  # in bps

# Calculate survival probability to maturity
ttm_years = (cds_maturity - calc_date) / 365.25
survival_prob = ford_hazard_curve.survivalProbability(cds_maturity)

print(f"\nCDS Metrics (100 bps coupon, maturity 2029-06-20):")
print(f"CDS PV: ${cds_npv:,.2f}")
print(f"Premium Leg PV: ${cds_coupon_leg:,.2f}")
print(f"Default Leg PV: ${cds_default_leg:,.2f}")
print(f"Par Spread: {cds_fair_spread:.2f} bps")
print(f"Survival Probability to Maturity ({ttm_years:.2f} years): {survival_prob:.4f}")

print("\n" + "="*80)
print("Problem 3 Complete!")

# 

# Problem 4

In [ ]:
PROBLEM 4: Derivation of Fixed Rate Bond PVs and DV01s in sympy (25 points)

print("\n" + "="*80)
print("PROBLEM 4: Derivation of Bond PVs and DV01s in sympy (25 points)")

# Define symbolic variables
T = sp.symbols('T', real=True, positive=True)
c = sp.symbols('c', real=True, positive=True)
y = sp.symbols('y', real=True, positive=True)

print("\n### Symbolic Variables Defined ###")
print("T = bond maturity (years)")
print("c = semi-annual coupon rate")
print("y = yield")

# Generic bond PV formula
bond_pv_eq = 1 + (c/2 - (sp.exp(y/2) - 1)) / (sp.exp(y/2) - 1) * (1 - sp.exp(-T*y))
print("\n### Generic Fixed Rate Bond PV Formula ###")
print(bond_pv_eq)

# Problem 4a: Zero Coupon Bond PV
print("\n### Problem 4a: Zero Coupon Bond PV ###")
zero_coupon_pv_eq = bond_pv_eq.subs(c, 0)
zero_coupon_pv_eq = sp.simplify(zero_coupon_pv_eq)
print("\nZero Coupon Bond PV (c=0):")
print(zero_coupon_pv_eq)
sp.pprint(zero_coupon_pv_eq)

# Create function from equation
zero_coupon_pv_func = sp.lambdify([T, y], zero_coupon_pv_eq)

# Plot Zero Coupon PV surface
try:
    plot_bond_function_surface(lambda c_val, T_val, y_val: zero_coupon_pv_func(T_val, y_val),
                              'Zero Coupon Bond PV')
    print("Zero Coupon PV surface plotted")
except Exception as e:
    print(f"Note: Surface plot requires matplotlib display: {e}")

# Problem 4b: Zero Coupon Bond DV01
print("\n### Problem 4b: Zero Coupon Bond DV01 ###")
zero_coupon_dv01_eq = sp.diff(zero_coupon_pv_eq, y)
zero_coupon_dv01_eq = sp.simplify(zero_coupon_dv01_eq)
print("\nZero Coupon Bond DV01 (derivative w.r.t. y):")
print(zero_coupon_dv01_eq)
sp.pprint(zero_coupon_dv01_eq)

# Create function
zero_coupon_dv01_func = sp.lambdify([T, y], zero_coupon_dv01_eq)

# Plot Zero Coupon DV01 surface
try:
    plot_bond_function_surface(lambda c_val, T_val, y_val: zero_coupon_dv01_func(T_val, y_val),
                              'Zero Coupon Bond DV01')
    print("Zero Coupon DV01 surface plotted")
except Exception as e:
    print(f"Note: Surface plot requires matplotlib display: {e}")

# Problem 4c: Interest Only Bond PV
print("\n### Problem 4c: Interest Only Bond PV ###")
# Interest Only = Generic Bond - Zero Coupon
interest_only_pv_eq = bond_pv_eq - zero_coupon_pv_eq
interest_only_pv_eq = sp.simplify(interest_only_pv_eq)
print("\nInterest Only Bond PV (Generic - Zero Coupon):")
print(interest_only_pv_eq)
sp.pprint(interest_only_pv_eq)

# Create function
interest_only_pv_func = sp.lambdify([c, T, y], interest_only_pv_eq)

# Plot Interest Only PV surface
try:
    plot_bond_function_surface(interest_only_pv_func, 'Interest Only Bond PV')
    print("Interest Only PV surface plotted")
except Exception as e:
    print(f"Note: Surface plot requires matplotlib display: {e}")

# Problem 4d: Interest Only Bond DV01
print("\n### Problem 4d: Interest Only Bond DV01 ###")
interest_only_dv01_eq = sp.diff(interest_only_pv_eq, y)
interest_only_dv01_eq = sp.simplify(interest_only_dv01_eq)
print("\nInterest Only Bond DV01 (derivative w.r.t. y):")
print(interest_only_dv01_eq)
sp.pprint(interest_only_dv01_eq)

# Create function
interest_only_dv01_func = sp.lambdify([c, T, y], interest_only_dv01_eq)

# Plot Interest Only DV01 surface
try:
    plot_bond_function_surface(interest_only_dv01_func, 'Interest Only Bond DV01')
    print("Interest Only DV01 surface plotted")
except Exception as e:
    print(f"Note: Surface plot requires matplotlib display: {e}")

# Problem 4e: Find coupon c* where IO PV = Zero Coupon PV
print("\n### Problem 4e: Coupon c* where Interest Only PV = Zero Coupon PV ###")
# Solve: interest_only_pv_eq = 0 (since IO - ZC = 0 means IO = ZC)
# Or equivalently: bond_pv_eq - zero_coupon_pv_eq = 0
# Which simplifies to: interest_only_pv_eq = 0

c_star_solution = sp.solve(interest_only_pv_eq, c)
print("\nSolving Interest_Only_PV(c*, y, T) = Zero_Coupon_PV(y, T)")
print("Equivalently: Interest_Only_PV(c*, y, T) = 0")
print("\nSolution for c*:")
if c_star_solution:
    c_star = c_star_solution[0]
    c_star = sp.simplify(c_star)
    print(c_star)
    sp.pprint(c_star)
else:
    print("No solution found")

print("\n" + "="*80)
print("Problem 4 Complete!")

# 

# Problem 5

In [ ]:
PROBLEM 5: LQD ETF Basket Analysis - Bucketed DV01 Risks (25 points)

print("\n" + "="*80)
print("PROBLEM 5: LQD ETF Basket Analysis (25 points)")

# Problem 5a: Load and explore LQD basket composition
print("\n### Problem 5a: Load and explore LQD basket composition ###")

# Load data
lqd_basket_df = pd.read_excel(data_path + 'lqd_basket_composition.xlsx')
lqd_symbology_df = pd.read_excel(data_path + 'lqd_corp_symbology.xlsx')

print(f"\nLQD Basket Composition: {len(lqd_basket_df)} bonds")
print(f"LQD Bond Symbology: {len(lqd_symbology_df)} bonds")

# Basic statistics
num_bonds = len(lqd_basket_df)
mean_notional = lqd_basket_df['face_notional'].mean()
median_notional = lqd_basket_df['face_notional'].median()

print(f"\nNumber of corporate bonds in LQD basket: {num_bonds}")
print(f"Average face notional per bond: ${mean_notional:,.2f}")
print(f"Median face notional per bond: ${median_notional:,.2f}")

# Ticker statistics
ticker_notionals = lqd_basket_df.groupby('ticker')['face_notional'].sum()
num_tickers = len(ticker_notionals)
mean_ticker_notional = ticker_notionals.mean()
median_ticker_notional = ticker_notionals.median()

print(f"\nNumber of unique tickers in LQD basket: {num_tickers}")
print(f"Average face notional per ticker: ${mean_ticker_notional:,.2f}")
print(f"Median face notional per ticker: ${median_ticker_notional:,.2f}")

# Yield statistics
print(f"\nYield-to-Maturity Statistics:")
print(f"Mean YTM: {lqd_basket_df['ytm'].mean()*100:.4f}%")
print(f"Median YTM: {lqd_basket_df['ytm'].median()*100:.4f}%")
print(f"Std Dev YTM: {lqd_basket_df['ytm'].std()*100:.4f}%")

# Problem 5b: Compute bond DV01 and basket contributions
print("\n### Problem 5b: Compute bond DV01 and basket DV01 contributions ###")

# Merge with symbology to get bond details
lqd_combined = lqd_basket_df.merge(lqd_symbology_df, on='isin', how='left')

# Calculate DV01 for each bond
bond_dv01_list = []
basket_dv01_list = []

for idx, row in lqd_combined.iterrows():
    try:
        # Create bond object
        bond = create_bond_from_symbology(row.to_dict())

        # Calculate duration and DV01
        ytm = row['ytm']
        duration = ql.BondFunctions.duration(bond, ytm, ql.Actual365Fixed(),
                                            ql.Compounded, ql.Semiannual)

        # DV01 per 100 face value
        bond_dv01 = duration / 100.0  # DV01 = Duration / 10000 * Price, assuming price ~100

        # Basket DV01 contribution
        basket_dv01 = bond_dv01 * row['face_notional'] / 100  # Scale by notional

        bond_dv01_list.append(bond_dv01)
        basket_dv01_list.append(basket_dv01)
    except Exception as e:
        bond_dv01_list.append(np.nan)
        basket_dv01_list.append(np.nan)

lqd_combined['bond_DV01'] = bond_dv01_list
lqd_combined['basket_DV01'] = basket_dv01_list

print(f"\nLQD Basket DataFrame with DV01 calculations:")
print(lqd_combined[['security', 'isin', 'ytm', 'face_notional', 'bond_DV01', 'basket_DV01']].head(10))

# Problem 5c: Aggregate by US Treasury buckets
print("\n### Problem 5c: Aggregate by US Treasury buckets ###")

# Group by underlying benchmark treasury
bucket_aggregation = lqd_combined.groupby('und_bench_tsy_isin').agg({
    'isin': 'count',  # Bond count
    'face_notional': 'sum',
    'basket_DV01': 'sum'
}).rename(columns={'isin': 'basket_count'})

print(f"\nAggregated LQD basket by US Treasury buckets:")
print(bucket_aggregation.to_string())

# Problem 5d: Display and plot aggregated data
print("\n### Problem 5d: Display and plot aggregated data ###")

# Load government bond symbology to get treasury details
govt_symbology_df = pd.read_excel(data_path + 'bond_symbology.xlsx')
govt_symbology_df = govt_symbology_df[govt_symbology_df['class'] == 'Govt'].copy()

# Merge with benchmark treasury info
bucket_aggregation = bucket_aggregation.reset_index()
bucket_aggregation = bucket_aggregation.merge(
    govt_symbology_df[['isin', 'security', 'ttm']],
    left_on='und_bench_tsy_isin',
    right_on='isin',
    how='left'
)

# Sort by TTM
bucket_aggregation = bucket_aggregation.sort_values('ttm')

print(f"\nCombined DataFrame with Treasury details:")
print(bucket_aggregation[['und_bench_tsy_isin', 'security', 'ttm', 'basket_count',
                          'face_notional', 'basket_DV01']].to_string(index=False))

# Find bucket with highest DV01
max_dv01_bucket = bucket_aggregation.loc[bucket_aggregation['basket_DV01'].idxmax()]
print(f"\nUS Treasury bucket with highest DV01 risk:")
print(f"Security: {max_dv01_bucket['security']}")
print(f"TTM: {max_dv01_bucket['ttm']:.2f} years")
print(f"Basket DV01: ${max_dv01_bucket['basket_DV01']:,.2f}")

# Create bar plots
fig_lqd = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Bond Count by Treasury Bucket',
                   'Face Notional by Treasury Bucket',
                   'Basket DV01 by Treasury Bucket'),
    vertical_spacing=0.12
)

# Plot 1: Bond count
fig_lqd.add_trace(
    go.Bar(x=bucket_aggregation['security'], y=bucket_aggregation['basket_count'],
           marker_color='lightblue', name='Bond Count'),
    row=1, col=1
)

# Plot 2: Face notional
fig_lqd.add_trace(
    go.Bar(x=bucket_aggregation['security'], y=bucket_aggregation['face_notional'],
           marker_color='lightgreen', name='Face Notional'),
    row=2, col=1
)

# Plot 3: Basket DV01
fig_lqd.add_trace(
    go.Bar(x=bucket_aggregation['security'], y=bucket_aggregation['basket_DV01'],
           marker_color='coral', name='Basket DV01'),
    row=3, col=1
)

fig_lqd.update_xaxes(tickangle=-45, row=1, col=1)
fig_lqd.update_xaxes(tickangle=-45, row=2, col=1)
fig_lqd.update_xaxes(tickangle=-45, row=3, col=1)
fig_lqd.update_yaxes(title_text='Count', row=1, col=1)
fig_lqd.update_yaxes(title_text='Face Notional ($)', row=2, col=1)
fig_lqd.update_yaxes(title_text='Basket DV01 ($)', row=3, col=1)
fig_lqd.update_layout(height=1000, showlegend=False, title_text='LQD Basket Analysis by Treasury Bucket')
fig_lqd.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/lqd_basket_analysis.html')
print("LQD basket analysis plots saved")

print("\n" + "="*80)
print("Problem 5 Complete!")

# 

# Problem 6

In [ ]:
PROBLEM 6: Nelson-Siegel Model for ORCL Curve (25 points)

print("\n" + "="*80)
print("PROBLEM 6: Nelson-Siegel Model for ORCL (25 points)")

# Problem 6a: Calibrate US on-the-run Treasury curve
print("\n### Problem 6a: Calibrate US on-the-run Treasury yield curve ###")

# Load on-the-run treasuries
govt_otr_df = pd.read_excel(data_path + 'govt_on_the_run.xlsx')
govt_otr_df['date'] = pd.to_datetime(govt_otr_df['date'])

# Filter for as_of_date
govt_otr_filtered = govt_otr_df[govt_otr_df['date'] == as_of_date].copy()

# Merge with symbology
govt_otr_combined = govt_symbology_df.merge(govt_otr_filtered, on='isin', how='inner')

print(f"\nOn-the-run Treasuries: {len(govt_otr_combined)} instruments")

# Calibrate Treasury curve
tsy_yield_curve = calibrate_yield_curve_from_frame(calc_date, govt_otr_combined)
tsy_curve_handle = ql.YieldTermStructureHandle(tsy_yield_curve)

# Get curve details
tsy_curve_df = get_yield_curve_details_df(tsy_yield_curve, curve_dates)

print(f"\nUS Treasury curve calibrated successfully")
print(f"Sample zero rates (first 5 tenors):")
print(tsy_curve_df.head())

# Plot Treasury curves
fig_tsy = make_subplots(
    rows=1, cols=2,
    subplot_titles=('US Treasury Zero Interest Rates', 'US Treasury Discount Factors')
)

fig_tsy.add_trace(
    go.Scatter(x=tsy_curve_df['TTM'], y=tsy_curve_df['Zero_Interest_Rate'] * 100,
               mode='lines', name='Zero Rate', line=dict(color='navy', width=2)),
    row=1, col=1
)

fig_tsy.add_trace(
    go.Scatter(x=tsy_curve_df['TTM'], y=tsy_curve_df['Discount_Factor'],
               mode='lines', name='Discount Factor', line=dict(color='darkgreen', width=2)),
    row=1, col=2
)

fig_tsy.update_xaxes(title_text='Time to Maturity (years)', row=1, col=1)
fig_tsy.update_xaxes(title_text='Time to Maturity (years)', row=1, col=2)
fig_tsy.update_yaxes(title_text='Zero Rate (%)', row=1, col=1)
fig_tsy.update_yaxes(title_text='Discount Factor', row=1, col=2)
fig_tsy.update_layout(height=400, showlegend=False, title_text='US Treasury Yield Curve')
fig_tsy.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/treasury_curve.html')
print("Treasury curve plots saved")

# Problem 6b: Prepare ORCL symbology and market data
print("\n### Problem 6b: Prepare ORCL symbology and market data ###")

# Load corporate bond data
corp_symbology_df = bond_symbology_df[bond_symbology_df['class'] == 'Corp'].copy()

# Filter for ORCL bonds
orcl_bonds = corp_symbology_df[
    (corp_symbology_df['ticker'] == 'ORCL') &
    (corp_symbology_df['cpn_type'] == 'FIXED') &
    (corp_symbology_df['amt_out'] > 100)
].copy()

# Merge with market data
bond_market_filtered = bond_market_df[bond_market_df['date'] == as_of_date].copy()
orcl_combined = orcl_bonds.merge(bond_market_filtered, on='isin', how='inner')

# Sort by maturity
orcl_combined = orcl_combined.sort_values('maturity')

print(f"\nORCL Bonds: {len(orcl_combined)} bonds")
print(f"\nORCL Bond Data (first 5):")
print(orcl_combined[['security', 'maturity', 'ttm', 'mid_yield']].head())

# Plot ORCL yields
fig_orcl_yields = go.Figure()
fig_orcl_yields.add_trace(go.Scatter(
    x=orcl_combined['ttm'],
    y=orcl_combined['mid_yield'],
    mode='markers',
    name='Market Yields',
    marker=dict(size=10, color='blue')
))
fig_orcl_yields.update_layout(
    title='ORCL Bond Yields by Time to Maturity',
    xaxis_title='Time to Maturity (years)',
    yaxis_title='Yield (%)',
    hovermode='closest'
)
fig_orcl_yields.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/orcl_market_yields.html')
print("ORCL market yields plot saved")

# Problem 6c: Calibrate Nelson-Siegel model
print("\n### Problem 6c: Calibrate Nelson-Siegel model for ORCL ###")

# Create bonds and weights for calibration
bonds_list, weights_list, market_prices = create_bonds_and_weights(
    orcl_combined.to_dict('records'), tsy_curve_handle
)

# Initial Nelson-Siegel parameters
initial_params = np.array([0.05, -0.02, 0.01, 2.0])

# Calibrate model
optimal_params = calibrate_nelson_siegel_model(
    initial_params, calc_date, bonds_list, weights_list,
    market_prices, tsy_curve_handle
)

print(f"\nOptimal Nelson-Siegel Parameters:")
print(f"β0 (level): {optimal_params[0]:.6f}")
print(f"β1 (slope): {optimal_params[1]:.6f}")
print(f"β2 (curvature): {optimal_params[2]:.6f}")
print(f"τ (decay): {optimal_params[3]:.6f}")

# Create calibrated credit curve
orcl_credit_curve = create_nelson_siegel_curve(calc_date, optimal_params)

print("ORCL smooth credit curve calibrated successfully")

# Problem 6d: Compute model prices, yields, and edges
print("\n### Problem 6d: Compute model prices, yields, and edges ###")

# Calculate model prices and yields
model_results = calculate_nelson_siegel_model_prices_and_yields(
    optimal_params, calc_date, bonds_list, weights_list,
    market_prices, tsy_curve_handle
)

# Add to dataframe
orcl_combined['modelPrice'] = model_results['model_prices']
orcl_combined['modelYield'] = model_results['model_yields']
orcl_combined['edgePrice'] = model_results['price_edges']
orcl_combined['edgeYield'] = model_results['yield_edges']

print(f"\nModel Results (first 5 bonds):")
print(orcl_combined[['security', 'mid_price', 'modelPrice', 'mid_yield',
                     'modelYield', 'edgePrice', 'edgeYield']].head())

# Problem 6e: Visualize results
print("\n### Problem 6e: Visualize calibration results ###")

# Plot model vs market prices
fig_orcl_prices = go.Figure()
fig_orcl_prices.add_trace(go.Scatter(
    x=orcl_combined['ttm'],
    y=orcl_combined['mid_price'],
    mode='markers',
    name='Market Price',
    marker=dict(size=10, color='blue')
))
fig_orcl_prices.add_trace(go.Scatter(
    x=orcl_combined['ttm'],
    y=orcl_combined['modelPrice'],
    mode='markers',
    name='Model Price',
    marker=dict(size=10, color='red', symbol='x')
))
fig_orcl_prices.update_layout(
    title='ORCL Bond Prices: Model vs Market',
    xaxis_title='Time to Maturity (years)',
    yaxis_title='Price',
    hovermode='x unified'
)
fig_orcl_prices.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/orcl_model_prices.html')

# Plot model vs market yields
fig_orcl_model_yields = go.Figure()
fig_orcl_model_yields.add_trace(go.Scatter(
    x=orcl_combined['ttm'],
    y=orcl_combined['mid_yield'],
    mode='markers',
    name='Market Yield',
    marker=dict(size=10, color='blue')
))
fig_orcl_model_yields.add_trace(go.Scatter(
    x=orcl_combined['ttm'],
    y=orcl_combined['modelYield'],
    mode='markers',
    name='Model Yield',
    marker=dict(size=10, color='red', symbol='x')
))
fig_orcl_model_yields.update_layout(
    title='ORCL Bond Yields: Model vs Market',
    xaxis_title='Time to Maturity (years)',
    yaxis_title='Yield (%)',
    hovermode='x unified'
)
fig_orcl_model_yields.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/orcl_model_yields.html')

# Plot yield edges
fig_orcl_edges = go.Figure()
fig_orcl_edges.add_trace(go.Scatter(
    x=orcl_combined['ttm'],
    y=orcl_combined['edgeYield'],
    mode='markers',
    name='Yield Edge',
    marker=dict(size=10, color='green')
))
fig_orcl_edges.add_hline(y=0, line_dash='dash', line_color='gray')
fig_orcl_edges.update_layout(
    title='ORCL Bond Yield Edges (Market - Model)',
    xaxis_title='Time to Maturity (years)',
    yaxis_title='Yield Edge (%)',
    hovermode='closest'
)
fig_orcl_edges.write_html('/home/user/Credit-Markets/UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam/orcl_yield_edges.html')
print("ORCL model visualization plots saved")

print("\n" + "="*80)
print("Problem 6 Complete!")

# 